### using all packages

In [10]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

### using module

In [11]:
include("HighDimMixedModels.jl")
import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [5]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = rand(rng, 9))
f = @formula(y ~  a + b + c + d)
contrasts = Dict( :a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)
df

,y,a,b,c,d
,Float64,Int64,Float64,String,Float64
1,0.585195,1,0.236782,d,0.752223
2,0.0773379,2,0.943741,e,0.314815
3,0.716628,3,0.445671,f,0.858522
4,0.320357,4,0.763679,d,0.698713
5,0.653093,5,0.145071,e,0.757746
6,0.236639,6,0.021124,f,0.419294
7,0.709684,7,0.152545,d,0.412607
8,0.557787,8,0.617492,e,0.454589
9,0.05079,9,0.481531,f,0.380933


### Construct object

#### 1.Number of variables indicates M,X,Z

In [6]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


Main.HighDimMM.highDimMixedModel{Float64}(y ~ a + b + c + d, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([0.236781883208121; 0.9437409715735081; … ; 0.617492416565387; 0.48153065407402607]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([1.0 0.0; 2.0 1.0; … ; 8.0 1.0; 9.0 0.0]), [0.0 0.7522231521093703; 0.0 0.31481548090434086; … ; 0.0 0.45458919106592477; 1.0 0.38093255230858825], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

#### 2.indices indicates M,X,Z

In [7]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, [2,4],3)

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


Main.HighDimMM.highDimMixedModel{Float64}(y ~ a + b + c + d, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([1.0; 2.0; … ; 8.0; 9.0]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([0.236781883208121 0.7522231521093703; 0.9437409715735081 0.31481548090434086; … ; 0.617492416565387 0.45458919106592477; 0.48153065407402607 0.38093255230858825]), [0.0 0.0; 1.0 0.0; … ; 1.0 0.0; 0.0 1.0], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

### 3.variable names indicates M,X,Z

In [12]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, "a", ["b","d"],"c")

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:12


Main.HighDimMM.highDimMixedModel{Float64}(y ~ a + b + c + d, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([1.0; 2.0; … ; 8.0; 9.0]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([0.236781883208121 0.7522231521093703; 0.9437409715735081 0.31481548090434086; … ; 0.617492416565387 0.45458919106592477; 0.48153065407402607 0.38093255230858825]), [0.0 0.0; 1.0 0.0; … ; 1.0 0.0; 0.0 1.0], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

### Optimization

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$
To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$

In [14]:
sigma, beta = HighDimMM.fit!(HMM, verbose = true, REML = true, alg = :LN_COBYLA)

OPTBL: starting point [2, 2]
got -39.83898 at [188229.14752, 496891.01255] after 265679 iterations (returned FORCED_STOP)


([188229.14751929764, 496891.0125509644], [0.01727688920494593, -0.2990299131352748, 0.871836712849253])

In [16]:
sigma

2-element Vector{Float64}:
 188229.14751929764
 496891.0125509644

After getting estimate of sigma, we estimate beta
$$
\hat{\beta}(\theta) = (X^T\Sigma^{-1}X)^{-1}X^T\Sigma^{-1}y
$$

In [15]:
beta

3-element Vector{Float64}:
  0.01727688920494593
 -0.2990299131352748
  0.871836712849253